In [1]:
from Bio import SeqIO
import gzip

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
import csv

In [4]:
import subprocess

## Functions

In [5]:
def rev_comp(seq):
    complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A','a': 't', 'c': 'g', 'g': 'c', 't': 'a'}
    return "".join(complement.get(base, base) for base in reversed(seq))

In [6]:
geneticcode = {'GGG':'G', 'GGA':'G', 'GGC':'G', 'GGT':'G','GAG':'E', 'GAA':'E', 'GAC':'D', 'GAT':'D','GCG':'A', 'GCA':'A', 'GCC':'A', 'GCT':'A',
               'GTG':'V', 'GTA':'V', 'GTC':'V', 'GTT':'V','AGG':'R', 'AGA':'R', 'AGC':'S', 'AGT':'S','AAG':'K', 'AAA':'K', 'AAC':'N', 'AAT':'N',
               'ACG':'T', 'ACA':'T', 'ACC':'T', 'ACT':'T','ATG':'M', 'ATA':'I', 'ATC':'I', 'ATT':'I','CGG':'R', 'CGA':'R', 'CGC':'R', 'CGT':'R',
               'CAG':'Q', 'CAA':'Q', 'CAC':'H', 'CAT':'H','CCG':'P', 'CCA':'P', 'CCC':'P', 'CCT':'P','CTG':'L', 'CTA':'L', 'CTC':'L', 'CTT':'L',
               'TGG':'W', 'TGA':'*', 'TGC':'C', 'TGT':'C','TAG':'*', 'TAA':'*', 'TAC':'Y', 'TAT':'Y','TCG':'S', 'TCA':'S', 'TCC':'S', 'TCT':'S','TTG':'L', 'TTA':'L', 'TTC':'F', 'TTT':'F'}

In [7]:
def translateDNA(seq):
    i = 0
    aa = ""
    while i <= len(seq)-3:
        try:
            aa += geneticcode[seq[i:i+3]]
            i +=3
        except:
            break
    return aa

In [8]:
alphabet = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P',
       'Q', 'R', 'S', 'T', 'V', 'W', 'Y', '*']

## Process raw reads

In [26]:
seq_dict = {}
with gzip.open('231212Lau/231212Lau_D23-238197-1_1_sequence.fastq.gz', "rt") as handle:
    for record in SeqIO.parse(handle, "fastq"):
        seq_dict[record.id.split('/')[0]] = str(record.seq)

print (len(seq_dict), 'total reads')

3820086 total reads


In [6]:
qual_scores = {}
with gzip.open('231212Lau/231212Lau_D23-238197-1_1_sequence.fastq.gz', "rt") as handle:
    for record in SeqIO.parse(handle, "fastq"):
        qual_scores[record.id.split('/')[0]] = record.letter_annotations["phred_quality"]


In [27]:
aligned_dict = {}
qual_aligned = {}
for entry in seq_dict:
    start_pos = seq_dict[entry].find('CTCAAATCTT') + 10
    aligned_dict[entry] = seq_dict[entry][start_pos:]
    qual_aligned[entry] = qual_scores[entry][start_pos:]

In [ ]:
filt_dict = {}
for entry in aligned_dict:
    seq = aligned_dict[entry]
    qual = qual_aligned[entry]
    for i in range(len(qual)):
        if qual[i] >= 30:
            continue
        else:
            qual.append('FLAG')
            

In [28]:
aa_dict = {}
for entry in filt_dict:
    aa_dict[entry] = translateDNA(filt_dict[entry])

In [29]:
aa_trim = {}
for entry in aa_dict:
    stop_pos = aa_dict[entry].find('*')
    if stop_pos != -1:
        aa_trim[entry] = aa_dict[entry][1:stop_pos]
    else:
        aa_trim[entry] = aa_dict[entry][1:]

In [18]:
binders_full = pd.read_csv('enrichment_ratio_extensions.csv')

In [19]:
seq_lib = list(binders_full['sequence'])

In [30]:
counts_1 = {}
#no_map = []
for seq in seq_lib:
    counts_1[seq] = 0
for read in aa_trim:
    if aa_trim[read] in seq_lib:
        counts_1[aa_trim[read]] += 1
    #else:
        #no_map.append(aa_trim[read])

In [1]:
#repeat for post-selection library

In [50]:
counts_1 = np.array(counts_naive['558'])
counts_2 = np.array(counts_post['1'])

In [51]:
freq_1 = counts_1/3820086
freq_2 = counts_2/3560082

In [52]:
enrichment_ratio = freq_2/freq_1

/home/culviner/common_py3/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.


In [67]:
enrichment_ratio_df = pd.DataFrame(index = seq_lib[1:], data=enrichment_ratio, columns=['er'])

In [60]:
enrichment_ratio_df.to_csv('enrichment_ratio_prelim.csv')

In [31]:
with open('enrichment_ratio_prelim.csv', 'w') as f:
    for key in counts.keys():
        f.write("%s,%s\n"%(key,counts[key]))
    #for key in counts_nomap.keys():
        #f.write("%s,%s\n"%(key,counts_nomap[key]))